In [4]:
import math
import numpy as np
import urllib
import json
from time import sleep
%matplotlib inline
import matplotlib.pylab as plt
from h3 import h3
import folium
import haversine
import os
import glob
import re

# code parameters

In [5]:
#get your polygon for the area of interest here
# http://geojson.io/#map=2/20.0/0.0
geoJson ={
        "type": "Polygon",
        "coordinates": [[
          [37.88122169005974,-122.31233596801756],
          [37.886099161825506,-122.28967666625977],
          [37.90364184038751,-122.28933334350585],
          [37.90689253397191,-122.26667404174805],
          [37.88833456175579,-122.24255561828612],
          [37.86319934044903,-122.23955154418945],
          [37.85601627346178,-122.23886489868164],
          [37.85662618370235,-122.21483230590819],
          [37.84456257747872,-122.21320152282715],
          [37.84090244155454,-122.25156784057617],
          [37.84530813845281,-122.25225448608398],
          [37.843206992760315,-122.26787567138672],
          [37.83364941345965,-122.26641654968262],
          [37.83175130707428,-122.28169441223145],
          [37.84029240124074,-122.28461265563963],
          [37.83507296120189,-122.31568336486818],
          [37.83873338640461,-122.31722831726074],
          [37.84395256743893,-122.31602668762207],
          [37.84307143291804,-122.30778694152833],
          [37.8404279661909,-122.30572700500487],
          [37.84144469537543,-122.2990322113037],
          [37.844494798834575,-122.3001480102539],
          [37.84517258247272,-122.30667114257812],
          [37.84862918212842,-122.30117797851561],
          [37.86082765045781,-122.30340957641602],
          [37.858794712577556,-122.3063278198242],
          [37.86035330330106,-122.30692863464354],
          [37.864893358000735,-122.30795860290527],
          [37.86428351616802,-122.31061935424805],
          [37.86170857279247,-122.31173515319826],
          [37.86164080990979,-122.31508255004883],
          [37.85852364995701,-122.3151683807373],
          [37.85818482027954,-122.3169708251953],
          [37.85872694701587,-122.32821464538576],
          [37.860759886765194,-122.3295021057129],
          [37.86374143030225,-122.31903076171875],
          [37.87471789187652,-122.32589721679688],
          [37.87634389525958,-122.31782913208008],
          [37.86963639998044,-122.31310844421387],
          [37.87078823552829,-122.30873107910156],
          [37.88162815837853,-122.31242179870607]
        ]]
      }
    

    
#enter your google api key here
googlePlacesKEY='';

#INPUT: resolution 1 to 15 where 1 is lowest resolution and 15 is highest
resolution=8;

#make sure you have a data folder in the same directory

# generating and vizualize the hexagons

In [6]:
hexagons=list(h3.polyfill(geoJson,resolution))
points=[h3.h3_to_geo_boundary(h) for h in hexagons] 
points_np=np.array(points)
midpoint=[np.mean(points_np[:,:,0]),np.mean(points_np[:,:,1])]
points_plotted=[i+[i[0]] for i in points];

m=plot_hexagons_in_poly(points_plotted,geoJson['coordinates'])
m

# [function] This takes a point and return the pois from Google places API

In [8]:
# This function does a nearby search on Google places API
# It takes a coordinate and the radius desired for search
# It returns a list of places with their features.
# for reference check: https://developers.google.com/places/web-service/search#PlaceSearchRequests
#
def fetchFromGooglePOI(point,radius,googlePlacesKEY):
    radiusInMeters=radius*1000;
    allDataFetched=False;
    data=[];
    url_='https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='+str(point[0][0])+','+str(point[0][1])+'&radius='+str(radiusInMeters)+'&key='+googlePlacesKEY;
    
    i=0;
    while(not allDataFetched):
        try:
            #sending google a request
            response = urllib.request.urlopen(url_)
            #getting back data in json format
            html = response.read()
            ##cleaned=html.replace("\n","")
            json_poi=json.loads(html)
            results=json_poi['results']
        except:
            return [],'URL EXCEPTION';

        if len(data)==0 and len(results)!=0:
            data=results;
        elif len(results)!=0:
            data=data+results;

            
        #check if there are more POIs to crawl   
        if 'next_page_token' in json_poi:# this means that there are more POIs to fetch, need to wait a little before sending another request
            sleep(2)
            url_='https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken='+str(json_poi['next_page_token'])+'&key='+str(googlePlacesKEY); 
        else:
            allDataFetched=True;
            


    if len(data)>=50:
        status="has More POIs";
    else:
        status="crawled All POIs";
    return data,status;



# Adaptive Scale Querying for Google API [takes some time]

In [ ]:
%%time
#This starts crawling Google API
#This code could take a while
#to test the code you can set below QueryLimit
#This will limit the number of sent queries and finish faster
QueryLimit=len(hexagons); #negative means query all points
data={}
inc=0;
while len(hexagons)>0:
    hexa=hexagons.pop();
    
    #check if the hexa was crawled already:
    if os.path.isfile('data/{}.json'.format(hexa)):
        print('{} was crawled already'.format(hexa))
        continue;
    
    #fetch the pois of this hexagon
    pointCoordinate=[tuple(h3.h3_to_geo(hexa))];
    radiusOfQuery=h3.edge_length(h3.h3_get_resolution(hexa))
    queryResults,status=fetchFromGooglePOI(pointCoordinate,radiusOfQuery,googlePlacesKEY);
    
    
    #adaptive querying if the hexagon has more POIs
    if status=='has More POIs':
            sub_hexagons=h3.h3_to_children(hexa,h3.h3_get_resolution(hexa)+1);
            hexagons=hexagons+list(sub_hexagons);
    #save it if crawling is complete
    elif status=='crawled All POIs':
        for place in queryResults:
            if place['place_id'] not in data:
                data[place['place_id']]=place;
        with open('data/{}.json'.format(hexa), 'w') as fp:
            json.dump(data, fp)
        data={};
    #alert the user if the URL wasn't fetched because of error
    elif status=='URL EXCEPTION':
        print(status)
        hexagons=hexagons+[hexa];
        sleep(3);
            
    
    inc+=1;
    # write about the progress of the code
    if inc%10:
        print ('{} hexagons remaining to crawl'.format(len(hexagons),len(data)))



64 hexagons remaining to crawl
8928308a827ffff was crawled already
69 hexagons remaining to crawl
8a28308a820ffff was crawled already
8a28308a8227fff was crawled already
8a28308a8207fff was crawled already
8a28308a821ffff was crawled already
8a28308a8217fff was crawled already
8a28308a822ffff was crawled already
8a28308a8237fff was crawled already
68 hexagons remaining to crawl
8a28308a8387fff was crawled already
8a28308a83a7fff was crawled already
8a28308a83b7fff was crawled already
8a28308a83affff was crawled already
8a28308a839ffff was crawled already
8a28308a838ffff was crawled already
8a28308a8397fff was crawled already
67 hexagons remaining to crawl
8a28308a82b7fff was crawled already
8a28308a82affff was crawled already
8a28308a8287fff was crawled already
8a28308a829ffff was crawled already
8a28308a828ffff was crawled already
8a28308a82a7fff was crawled already
8a28308a8297fff was crawled already
66 hexagons remaining to crawl
8a28308a8377fff was crawled already
8a28308a8347fff w

# Viewing the crawled hexagons

In [8]:
c_points=[];
for file in glob.glob("data/*.json"):
    hexa=file.replace('data/','').replace('.json','')
    c_points=c_points+[h3.h3_to_geo_boundary(hexa)];
    
m=plot_hexagons_in_poly(c_points,geoJson['coordinates'])
m

# Viewing the crawled data: viewing places names

In [7]:
places_names=[];
for file in glob.glob("data/*.json"):
    with open(file) as f:
        data=json.load(f)
    for k in data:
        places_names.append(data[k]['name'])
        
print('there are {} places in the crawled area, below are their names'.format(len(places_names)));
print('----------')
for i in places_names:
        print(i)

there are 12406 places in the crawled area, below are their names
----------
312-350 62nd St
Noodle Theory | Rockridge
Claremont Rug Company
Bank of America Financial Center
Dana Eugenia Locke
Journeys Inward CranioSacral Therapy and Integrative Bodywork - Oakland
Claremont 2
Ellie Waxman, LMFT
The Breema Center
ATM (Bank of America)
Janet A. King, MFC
Bank of America Mortgage
Linda N. Kuwatani, MS
Marian T. Okamura LCSW
Moxley K Mona PhD
Hennigan Patricia MD
Josephine S. Gallup, PSYD
Brandt Mayflower D
Fernando Moratinos. Massage Therapist
Oakland
Claremont Av:College Av
Rogers Lynn S
Doug Jameson, Psy.D.
Claremont Av:College Av
Kensington
Kensington Plastering
2600-2698 Eighth Street
Applied Anatomy Institute
FB Design Build
Argyle Design
Consolidated Printers, Inc.
Hopscotch Press Inc
Devi Dutta Architecture
Standard Fare
La Noisette Sweets
Cakes Made By Me
Endorfin
Baron Baking
Tofu Yu
EnChroma, Inc.
Sean Gelbaugh
The Berkeley Kitchens
Ncubate Studios
Ringlight Studios
Kevin E. Sto

Arlington Travel
Mechanics Bank
Kramer Sandra
Raphael Deborah
Jones Bart
Mink Jeff
Arlington Av:#259
Tom Diedrich Carpentry
Morphopoulos Hercules D DDS
Inca Arechiga
Eleanor Gayle Mft & Consultant
Kensington Style House
Zarrin Ferdowsi DDS
Welden & Welden
Robert N. Dreyfuss, OMD
Mercurio Dental Arts
Dr. Ferdowsi DDS
Felicia Cole Consulting Services
Archie's Tax Services
Bemesderfer Sandra PHD
Oswald Engineering Associates Inc
Jenkins James PHD
Morrison Carol a PhD
Kensington Nail Spa
Arlington Av:Oberlin Av
Mesh Motion Inc
1026-1098 Carleton St
Far Leaves Tea
True Love Tattoo
Carleton Associates
Orozco's Tires
Oceanworks II
Dave's Record Shop
Covenant Worship Center
Hamilton & Co Architecture
Echo Photographics
Duran Leandro H
Hooven & Hooven Porcelain
Proper Attire US
Zinzin
Zion Baptist Church
The Event Center
Oceanworks Berkeley
Northern California District Council
Alward Construction
Berkeley
Herrera Evelyn
Carleton Street Partners
Berkeley Chess School
Sonia Gill
Orozco's Tires
No

Amphitheater
Americaone
Trackers
University Av:Bait Shop
The Wallace Alexander Gerbode Foundation
Berkeley sportsmans center
Berkeley marina sports men center
California Dawn Sportfishing
Golden Eye 2000 Sport Fishing
QM Travels Sailing Experiences
Cal Adventures West Dock
Cal Adventures East Dock
925-901 Pardee St
L.J. Kruse Company
Kruse Shipping & Receiving
Berkeley
Ohlone Greenway
Mizuki Auto - Smog Check
Schonfield Consulting
Martin Luther King Jr Way:Hearst Av
No 1 Gas
Martin Luther King Jr Way:Hearst Av
Berkeley
Berkeley
Michael W. Haas, MFT
Designs With Nature
Gil Faust - Mortgage Banker - NMLS 283798
Rosalyn Rivkin, LCSW
Euclid Av:Acacia Av
Euclid Av:Grizzly Peak Blvd
Euclid Ave & Grizzly Peak Blvd
Euclid Av:Acacia Av
Kelling Investment Advisors
Euclid Ave. at Acacia Ave.
Grizzly Peak Blvd:Creston Rd (North-West Jctn)
Grizzly Peak Blvd:Creston Rd (North-West Jctn)
Grizzly Peak Blvd:Creston Rd (North-West Jctn)
Wilson Beverly
Graetch Landscaping
Berkeley Hills
Berkeley
Kristen 

Arlene's Cleaners
Harmony Beauty Boutique
Smooth & Sassy
j. Brooker Studio
Deirdre O'Malley Makeup & Derm
Palm Açai Café
College Av:Ashby Av
Versailles Nail Spa
College Av:Ashby Av
Ashby Av:College Av
Stewart M Baker Inc
Ashby Av:College Av
Sidell Pakravan
Tulanian
Palm + Perkins
Dream Fluff Donuts
2530-2580 Bancroft Way
United States Postal Service
Café Milano
Yogurtland
Gypsy's Trattoria Italiana
Bancroft Clothing Co
Café Durant
Steve's Korean Bar-B-Que
KINGPIN DONUTS
Chang Luong Restaurant
Bancroft Center Co
Tivoli Caffe
Mandarin House
Kimchi Garden
IB's Berkeley
Pancho's Mexican Grill
Shihlin Taiwan Street Snacks
UC Berkeley 1
Phone Repair Hut
Berkeley
El Granada Apartments
DAN GUITAR MUSIC - MUSIC BY THE HOUR
Jamba Juice
Johnston's Market
Millennials
Bear's Ramen House
Diyar Mediterranean Cuisine
Sweetheart Café
GBC
Thai Basil
Cal Gyros
Katsumi Sushi
Daruma Japanese Restaurant
Tea Press Cafe
Boba Ninja
Cal Gyro Mediterranean
Tea Press
Lotus house
Vincent Bail Bonds
Crowdbotics
Sum

Boat Launch Ramps
Berkeley Marina Boat Launch Ramps
Berkeley Marina
2007-2001 Emerson St
Flacos
Black & White Store
Antiques & Modern
Trackers Earth: Bay Area
Jack's Antiques
MASTER FADE Audio/Music Services
Swipe USA, LLC
ATM
African Braiding-Le Balafon
Joanne B. Somerville, Chiropractor
Berkeley
2300-2398 Sixth Street
Frutiger Elizabeth A MD
LifeLong Medical Care: Administrative Offices
Lighting Systems
California Physicians Alliance
Bonita Home Inc
Best House Services
Christy Bertani Bookkeeping
Joshua F. Sandeman, FNP
Ellen T. Vargas, LCSW
Livingston Shirley A DDS
6th St:Channing Way
6th St:Channing Way
Bone Emily
6th St:Channing Way
Berkeley
6211-6201 Claremont Ave
1st United Credit Union
Great Clips
Philz Coffee
Oakland
Berkeley
Kinsey Consulting Services
Spring Mansion
California Environmental
CEI Today
Solution First
Arlington Av:Thousand Oaks Blvd
Arlington Av:Thousand Oaks Blvd
Berkeley Hills
800-898 Grayson St
Berkeley Advanced Biomaterials
Eastern Sun Traditional Chinese Me

# plotting function

In [1]:
def plot_hexagons_in_poly(hexagons,poly):
    points_np=np.array(hexagons)
    midpoint=[np.mean(points_np[:,:,0]),np.mean(points_np[:,:,1])]
    points_plotted=[i+[i[0]] for i in hexagons];
    m = folium.Map(midpoint,tiles='stamenterrain', zoom_start=12)
    #folium.ColorLine(geoJson['coordinates']).add_to(m)
    folium.features.PolyLine(poly,color='red',weight=7).add_to(m)
    folium.features.PolyLine(points_plotted,weight=3).add_to(m)
    return m;